In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [8]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
pulsed=True
p_folder="../../plots/run_"+str(r_n)+"/"

In [9]:
# helper functions
def choose_bar(df,layer, strip):
    df=df[df["strip"]==strip]
    df=df[df["layer"]==layer]
    return df

def import_data(calibration_folder, data_folder, run_number, is_it_pulsed=False):
    pedestal_df=pd.read_csv(calibration_folder+"pedestal.csv", sep=',')
    mip_df=pd.read_csv(calibration_folder+"mip.csv", sep=',')
    if is_it_pulsed==True:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+"_pulse.csv", sep=',')
    else:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+".csv", sep=',')
        
    return pedestal_df,mip_df,run_df

In [ ]:
def select_bars_with_data(run_df, pedestal_df=None, mip_df=None, subtract_pedestal=False, convert_mip=False):
    data_df=[]
    
    layers=np.arange(1,20)
    strips=np.arange(0,12)
    
    for layer in layers:
        for strip in strips:
            df_slice=choose_bar(df,layer,strip)
            pedestal_slice=choose_bar(pedestals,layer,strip)
            mip=choose_bar(mips,layer,strip)
            if not pedestal_slice.empty:
                df_slice=df_slice[df_slice["adc_sum_end0"]>1.2*pedestal_slice.iloc[0,-2]] # there could be a switch whether we want to
                df_slice=df_slice[df_slice["adc_sum_end1"]>1.2*pedestal_slice.iloc[1,-2]] # require both ends to register the hit

                df_slice.loc[:,"adc_sum_end0"]-=pedestal_slice.iloc[0,-2] # subtracting pedestals
                df_slice.loc[:,"adc_sum_end1"]-=pedestal_slice.iloc[1,-2]

                df_slice.loc[:,"adc_sum_end0"]*=(4.66/mip.iloc[0,-1]) # converting to energy
                df_slice.loc[:,"adc_sum_end1"]*=(4.66/mip.iloc[1,-1])
                
            confirmed_data.extend(df_slice.values.tolist())
    confirmed_df=pd.DataFrame(confirmed_data, columns=['event', 'adc_sum_end0', 'layer', 'strip', 'adc_sum_end1'])
    events_left=confirmed_df.event.unique()
    print("Initial pedestal-based selection performed.")
    print("Events left: "+str(len(events_left)))
    print("\n")
    return confirmed_df